In [1]:
# 1. Import libraries
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib


In [2]:
# 2. Load data
X_train = np.load("../data/processed/train_test/X_train.npy")
y_train = np.load("../data/processed/train_test/y_train.npy")
X_val = np.load("../data/processed/train_test/X_val.npy")
y_val = np.load("../data/processed/train_test/y_val.npy")
X_test = np.load("../data/processed/train_test/X_test.npy")
y_test = np.load("../data/processed/train_test/y_test.npy")


In [3]:
# 3. Convert to PyTorch tensors and DataLoader
batch_size = 64

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                            torch.tensor(y_val, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [4]:
# 4. Define MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [5]:
# 5. Initialize model, optimizer, and loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim=X_train.shape[1]).to(device)

pos_weight = torch.tensor([len(y_train) / sum(y_train) - 1]).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


In [6]:
# 6. Training loop with early stopping
best_val_loss = float("inf")
patience = 5
counter = 0

for epoch in range(30):
    model.train()
    train_losses = []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)
    avg_val_loss = np.mean(val_losses)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "../models/mlp_model_final.pt")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break
    scheduler.step()


Epoch 1, Train Loss: 0.3952, Val Loss: 0.3115
Epoch 2, Train Loss: 0.2279, Val Loss: 0.2831
Epoch 3, Train Loss: 0.2008, Val Loss: 0.2476
Epoch 4, Train Loss: 0.1905, Val Loss: 0.2298
Epoch 5, Train Loss: 0.1840, Val Loss: 0.2206
Epoch 6, Train Loss: 0.1831, Val Loss: 0.2344
Epoch 7, Train Loss: 0.1796, Val Loss: 0.2317
Epoch 8, Train Loss: 0.1778, Val Loss: 0.2152
Epoch 9, Train Loss: 0.1777, Val Loss: 0.2230
Epoch 10, Train Loss: 0.1768, Val Loss: 0.2210
Epoch 11, Train Loss: 0.1769, Val Loss: 0.2204
Epoch 12, Train Loss: 0.1757, Val Loss: 0.2184
Epoch 13, Train Loss: 0.1761, Val Loss: 0.2219
Early stopping triggered.


In [7]:
# 7. Evaluate on test set
model.load_state_dict(torch.load("../models/mlp_model_final.pt"))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(yb.numpy())

pred_labels = (np.array(all_preds) > 0.5).astype(int)

acc = accuracy_score(all_labels, pred_labels)
prec = precision_score(all_labels, pred_labels)
rec = recall_score(all_labels, pred_labels)
f1 = f1_score(all_labels, pred_labels)
cm = confusion_matrix(all_labels, pred_labels)

print("=== Test Evaluation ===")
print(f"Accuracy : {acc}")
print(f"Precision: {prec}")
print(f"Recall   : {rec}")
print(f"F1 Score : {f1}")
print("Confusion Matrix:\n", cm)


=== Test Evaluation ===
Accuracy : 0.9198658218412226
Precision: 1.0
Recall   : 0.9118491184911849
F1 Score : 0.9538923439845592
Confusion Matrix:
 [[ 244    0]
 [ 215 2224]]


C:\Users\ivanr\AppData\Local\Temp\ipykernel_11628\3055010158.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/mlp_model_final.